## Разница между методом и функцией

In [ ]:
# model = LinearRegression( )
# model.fit(X, y)
# model.coef_ 

# another_model = LinearRegression( )
# another_model.fit(Z, y)
# another_model.coef_ 

# kartoshka.jarit( )   метод
# jarit(kartoshka)     функция

In [4]:
import numpy as np

x = np.array([1,2,4])
x.sum(), sum(x)

(7, 7)

In [ ]:
class Killer:
    
    def f(x):
        pass
    
model = ...

In [60]:
print(f"было {x} игроков")

было [6 0 1 8 2 9 3 7 5 4] игроков


In [68]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
print(model)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


## Немного numpy

In [29]:
x = np.arange(10)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [30]:
np.random.shuffle(x)
x

array([6, 0, 1, 8, 2, 9, 3, 7, 5, 4])

In [73]:
y = np.ones(5)
y

array([1., 1., 1., 1., 1.])

In [74]:
z = np.zeros(5)
z

array([0., 0., 0., 0., 0.])

In [81]:
np.concatenate((y,z))

array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.])

In [127]:
np.random.choice(x)

6

In [147]:
alive = np.array([1,0,1,0,1,1])
target = np.arange(6)
target[alive == 1]

array([0, 2, 4, 5])

## Классы

* Пример с комплесными числами: https://github.com/hse-econ-data-science/eds_spring_2020/blob/master/sem04_class/sem_04_182.py

In [202]:
class Killer:
    
    # конструктор (вызывается один раз при создании нового экземпляра класса)
    # self - способ сослаться на самого себя 
    def __init__(self, n, verbose=False):
        
        # атрибуты класса 
        self.n = n
        self.verbose = verbose
        self.murder_target = np.arange(n)         # раздача целей по конвертам 
        self.alive = np.ones(n, dtype="uint8")    # жив ли игрок
        self.reward = np.zeros(n, dtype="uint8")  # очки, набранные игроком
        
        # сколько раз высшая сила уберегла
        self.savegod = np.zeros(n, dtype="int64")
        
        # функций, которые надо запустить при инициализции
        self.__create_game__()
        
    def __repr__(self): # описание экземпляра
        return f"Killer({self.n})"
    
    def __str__(self): # более человечное описание
        return f"Игра в киллера на {self.n} человек"
    
    # определяем своё собственное сложение 
    def __add__(self, other):
        n = self.n + other.n
        game = Killer(n)
        game.murder_target = np.concatenate((self.murder_target, other.murder_target + self.n))
        return game
    
    # дополнение текущей игры новыми людьми
    def add(self, other):
        self.murder_target = np.concatenate((self.murder_target, other.murder_target + self.n))
        self.n = self.n + other.n
            
    def __create_game__(self):
        np.random.shuffle(self.murder_target)
    
    def is_suicide(self, i):
        return self.murder_target[i] == i
    
    def is_matching(self, i):
        return self.murder_target[self.murder_target[i]] == i
    
    def step(self):
        # индекс убийцы выбрали случайно только среди живых
        killer_index = np.random.choice(self.murder_target[self.alive == 1])  
        killer_target = self.murder_target[killer_index]     # индекс жертвы 
        
        self.alive[killer_target] = 0  # пометили что жертва мертва
        self.reward[killer_index] += 1 # отдали фраг в награду
        
        # [2, 1, 3, 0 ]  2 игрок делает ход
        # [2, 1, 0, -1]  3 умер, его конверт переехал ко 2 
        self.murder_target[killer_index] = self.murder_target[killer_target]
        # self.murder_target[killer_target] = -1
        
        # считаем сколько раз чувака уберегла высшая сила от смерти 
        self.savegod[self.murder_target[killer_target]] += 1
        
        if self.verbose:
            print(f"Ход делает игрок {killer_index}")
            print(self.murder_target)
            print(self.alive)
            print(self.reward)
            print()
        
    def play_game(self):
        if self.verbose:
            print(f"Стартовая позиция:")
            print(self.murder_target)
            print(self.alive)
            print(self.reward)
            print()
        
        while np.sum(self.alive) > 0:
            self.step()
        
        if self.verbose:
            print("Игра завершилась")
            print(f"Победил игрок {np.argmax(self.reward)} со счётом {np.max(self.reward)}")
            print(f"Неубиваемым оказался игрок {np.argmax(self.savegod)} со счётом {np.max(self.savegod)}")
        
    def get_stat(self):
        stat = dict()
        stat['winner'] = np.argmax(self.reward)
        stat['winner_reward'] = np.max(self.reward)
        
        stat['savegod'] = np.argmax(self.savegod)
        stat['savegod_point'] = np.max(self.savegod)
        return stat
    

In [129]:
game1 = Killer(10)
game1.murder_target

array([3, 9, 2, 8, 5, 0, 4, 7, 6, 1])

In [130]:
game2 = Killer(5)
game2.murder_target

array([4, 0, 2, 3, 1])

In [123]:
game = game1 * game2
game

Killer(15)

In [116]:
game.murder_target

array([ 3,  0,  6,  1,  4,  5,  2,  7,  8,  9, 11, 13, 12, 10, 14])

In [117]:
game1.add(game2)

In [118]:
game1.murder_target

array([ 3,  0,  6,  1,  4,  5,  2,  7,  8,  9, 11, 13, 12, 10, 14])

In [200]:
game = Killer(100)
game.play_game()

Игра завершилась
Победил игрок 71 со счётом 7
Неубиваемым оказался игрок 43 со счётом 7


Случайная величина $X$ - максимально неуловимый Джо. Это игрок, киллера которого постоянно убивают и он постоянно меняется. То есть $X$ - это максимольно разнок число киллеров для фиксированной жертвы. 

* $E(X)$ - ?  
* Какова верятность, что победитель и неубиваемый совпадают? 

In [204]:
n_obs = 1000

stat = [ ]
for i in range(n_obs):
    game = Killer(100)
    game.play_game()
    stat.append(game.get_stat())

In [205]:
import pandas as pd 

df = pd.DataFrame(stat)
df.head()

,winner,winner_reward,savegod,savegod_point
0,79,7,6,5
1,62,8,88,7
2,45,8,76,8
3,42,9,42,7
4,20,9,74,7


In [206]:
# E(X) 
df['savegod_point'].mean()

6.682

In [207]:
df['winner_reward'].mean()

6.765

In [208]:
(df['savegod_point'] - df['winner_reward']).mean()

-0.083

In [212]:
import scipy.stats as sts
sts.ttest_rel(df['savegod_point'], df['winner_reward'])

Ttest_relResult(statistic=-1.3543253188690567, pvalue=0.17593885535733933)

In [213]:
# P(winner = savegod)
(df['winner'] == df['savegod']).mean()

0.193

# Наследование

In [231]:
class Killer_organizer(Killer):
    
    def __init__(self, n):
        Killer.__init__(self, n)
        self.__drop_suicide__()
        self.shuffle = 0
        
#     # борьба с суицидами 
#     def __drop_suicide__(self):
#         while sum(self.murder_target == np.arange(self.n)) > 0:
#             np.random.shuffle(self.murder_target)

#     # борьба с мэтчингами тем же дедовским способом: перемешиванием!
#     def __drop_matching__(self):
#         matching_cnt = sum([self.is_matching(item) for item in self.murder_target])
#         suicide_cnt = sum(self.murder_target == np.arange(self.n))
#         while (matching_cnt > 0)|(suicide_cnt  > 0):
#             self.shuffle += 1
#             np.random.shuffle(self.murder_target)        
        
    def step_organizer(self):
        

        # индекс убийцы выбрали случайно только среди живых
        killer_index = np.random.choice(self.murder_target[self.alive == 1])  
        killer_target = self.murder_target[killer_index]     # индекс жертвы 
        
        self.alive[killer_target] = 0  # пометили что жертва мертва
        self.reward[killer_index] += 1 # отдали фраг в награду
        
        # [2, 1, 3, 0 ]  2 игрок делает ход
        # [2, 1, 0, -1]  3 умер, его конверт переехал ко 2 
        self.murder_target[killer_index] = self.murder_target[killer_target]
        # self.murder_target[killer_target] = -1
        
        # считаем сколько раз чувака уберегла высшая сила от смерти 
        self.savegod[self.murder_target[killer_target]] += 1
        

        
        
#     def play_game(self):
#         if self.verbose:
#             print(f"Стартовая позиция:")
#             print(self.murder_target)
#             print(self.alive)
#             print(self.reward)
#             print()
        
#         while np.sum(self.alive) > 0:
#             self.step()
        
#         if self.verbose:
#             print("Игра завершилась")
#             print(f"Победил игрок {np.argmax(self.reward)} со счётом {np.max(self.reward)}")
#             print(f"Неубиваемым оказался игрок {np.argmax(self.savegod)} со счётом {np.max(self.savegod)}")
    

In [270]:
game = Killer_organizer(100)

In [226]:
game.g

5